In [1]:
# Initialization

import math
from datetime import datetime
import openpyxl as opxl
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import model as ty
import proc as pr
import matplotlib.ticker as mtick
import sklearn.metrics

import scipy.constants as scc

import sys
sys.path.append('../RD-systems-and-test-benches')

import thermal.SK_standard as nm

import seaborn as sns

import utils.plot_functions as RD_pfun
import repo_config as rconfig

from IPython.core.display import HTML

import heat_transfer as bht

import fluids as fds
import ht 

import general as gen

import os

import scipy.integrate as integrate
import scipy.optimize as sco

import networkx as nx

import plotly.graph_objects as go

import plot_functions_here as pfun

import plotly.io as pio

from itertools import product
from itertools import chain

from tqdm.notebook import tqdm

import copy

from concurrent.futures import ProcessPoolExecutor

In [2]:
folder_path = r'G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-perf-1Dmodel'
geometry_path = os.path.join(folder_path, 'Panels\\Panel_V4.1I.xlsx')
hypotheses_path = folder_path + '\\' + 'Hypotheses\\Model_hypotheses_calibrated.xlsx'

# panelSpecs = pr.import_geometry(geometry_path)
# hyp = pr.create_dict_from_excel(hypotheses_path,'Main')

# steadyStateConditions_df = pd.read_excel(condi_path,header=2) ## Les deux première lignes ne sont pas à prendre en compte
# stepConditions = steadyStateConditions_df.to_dict(orient='records')

In [3]:
recap = pd.read_excel(r'G:\Drive partagés\Cercle Hard\R&D\Thermal performance tests PVT\Recap_RD_TUV_SSI.xlsx')
recap = recap.drop(index=0)
recap.dropna(how='all', inplace=True)

groups = {key: group for key, group in recap.groupby('Design')}

In [4]:
triplets = [
            {'design_name' : 'V4.411(NI)',
             'test_type' : 'SK'},
            {'design_name' : 'V4.411(NI)',
                'test_type' : 'Dark front'},
            {'design_name' : 'V4.412(F)',
                'test_type' : 'SK'},
            {'design_name' : 'V4.412(F)',
                'test_type' : 'Dark front'}
                ]

for triplet in triplets:
             
    triplet['panelSpecs'] = pr.import_geometry(os.path.join(f'{folder_path}\\Panels', f"Panel_{triplet['design_name']}.xlsx"))

    # if triplet['design_name'] == 'V4.1NI':
    #     triplet['panelSpecs']['part2']['lambd_abs_ins'] = 0.
    #     triplet['panelSpecs']['part2']['lambd_tube_ins'] = 0.

    #     pr.update_panelSpecs_after_modif(triplet['panelSpecs'])
        
    triplet['hyp'] = pr.create_dict_from_excel(hypotheses_path,'Main')
    triplet['hyp']['method_h_top_g_exchanger'] = "forced_with_coeff_steps"
    triplet['steadyStateConditions_df'] = groups[triplet['design_name']].loc[groups[triplet['design_name']]['Type'] == triplet['test_type']].copy()
    pr.adjust_steadyStateConditions(triplet['steadyStateConditions_df'],triplet['hyp'],convert_celsius=True)

    triplet['steadyStateConditions_df'] = triplet['steadyStateConditions_df'].loc[(triplet['steadyStateConditions_df']['u'] != " ")]
    triplet['steadyStateConditions_df']['u'] = triplet['steadyStateConditions_df']['u'].astype(float)
    triplet['steadyStateConditions_df'] = triplet['steadyStateConditions_df'].loc[(triplet['steadyStateConditions_df']['u'] >= 0.1)]

    triplet['steadyStateConditions_df'].reset_index(drop=True,inplace=True)

### Implementation

In [5]:
mapping_parameters_bounds = {

    'V4.411(NI)' : {
        'panelSpecs': {
            'flat' : {
                    'lambd_air' : (0.05e-3,2.e-3),
            }
        }
    },

    'V4.412(F)' : {
        'panelSpecs': {
            'flat' : {
                    'lambd_air' : (0.05e-3,2.e-3),
            }
        }
    },

    'shared_designs' : {
        'panelSpecs': {
            'pv' : {
                    'tau_g' : (0.85, 0.95),
                    'eta_nom' : (0.21, 0.22),
                    # 'alpha_g' : (0.03,0.09),
                    # 'alpha_PV' : (0.87, 0.93),
                    # 'eps_PV' : (0.85, 0.95),
                    # 'eps_g' : (0.85, 0.95),
            },
            'flat' : {
                    # 'l_c' : (0.1e-3,0.6e-3),
                    'eps_tube' : (0.01,0.40),
                    # 'eps_ins' : (0.25,0.60),
            },
        }
    },

    'hyp': {
        # 'coeff_h_top_forced_range0' : (0.5, 3.),
        # 'coeff_h_top_forced_range1' : (0.5, 3.),
        # 'coeff_h_top_forced_range2' : (0.5, 3.),
    },
}

# Function to recursively flatten the parameter bounds
def flatten_parameters_bounds(d, parent_keys=[]):
    items = []
    for k, v in d.items():
        if isinstance(v, dict):
            items.extend(flatten_parameters_bounds(v, parent_keys + [k]))
        else:
            items.append((parent_keys + [k], v))
    return items

# Flatten the parameter bounds and extract parameter paths and bounds
flattened_parameters = flatten_parameters_bounds(mapping_parameters_bounds)
parameter_paths = [param[0] for param in flattened_parameters]
bounds = [param[1] for param in flattened_parameters]

mapping_parameters = {tuple(flattened_parameters[j][0]): j for j in range(len(flattened_parameters))}

In [6]:
def flatten_dict(d, parent_key=()):
    """ Recursively flattens a nested dictionary. Keys become tuples of the path."""
    items = []
    for k, v in d.items():
        new_key = parent_key + (k,)
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [7]:
def update_specs(params, design, panelSpecs):
    for key in mapping_parameters_bounds[design]['panelSpecs']:
        if key in list(set(panelSpecs['decomp'].values())) + ['pv']:
            zones = list(panelSpecs['decomp'].keys()) + ['pv'] if key == 'pv' else [_ for _, value in panelSpecs['decomp'].items() if value == key]
            for z in zones:
                for subkey in mapping_parameters_bounds[design]['panelSpecs'][key]:
                    panelSpecs[z][subkey] = params[mapping_parameters[(design, 'panelSpecs', key, subkey)]]

                    if subkey == 'eps_tube':
                        for i in range(1,8):
                            panelSpecs[f'part{i}']['eps_tube'] = params[mapping_parameters[(design, 'panelSpecs', key, subkey)]]
                            panelSpecs[f'part{i}']['eps_abs'] = params[mapping_parameters[(design, 'panelSpecs', key, subkey)]]
        else:
            panelSpecs[key] = params[mapping_parameters[(design, 'panelSpecs', key)]]

def update_panelSpecs(params, design_name, panelSpecs):
    update_specs(params, design_name, panelSpecs)
    update_specs(params, 'shared_designs', panelSpecs)
    pr.update_panelSpecs_after_modif(panelSpecs)

def update_hyp(params, hyp):

    for key in mapping_parameters_bounds['hyp']:
        hyp[key] = params[mapping_parameters[('hyp', key)]]

In [8]:
def simulation_function(params, triplet):

    panelSpecs = copy.deepcopy(triplet['panelSpecs'])
    hyp = copy.deepcopy(triplet['hyp'])
    steadyStateConditions_df = triplet['steadyStateConditions_df']

    update_panelSpecs(params, triplet['design_name'], panelSpecs)
    update_hyp(params, hyp)

    # Call the simulation function
    df_res, list_res = ty.simu_steadyStateConditions(panelSpecs, hyp, steadyStateConditions_df)
    return df_res


# Global counter for objective function calls
call_count = 0

# def callback_function(xk):
#     print(f"Iteration {len(iteration_history)}: params = {xk}")
#     iteration_history.append(xk.copy())

def objective_function(params, triplets):

    global call_count
    call_count += 1

    total_error = 0.0

    for triplet in triplets:

        df_res = simulation_function(params, triplet)

        # Compute the error
        error = np.sqrt(((df_res['Qdot_tube_fluid'] - triplet['steadyStateConditions_df']['Qdot']) ** 2).mean())
        total_error += error

    return total_error

def run_optimization(initial_guess, bounds, triplets):
    result = sco.minimize(
        objective_function,
        x0=initial_guess,
        args=(triplets,),  # Ensure you're passing the right triplet or list of triplets
        bounds=bounds,
        method='L-BFGS-B',
        options={'disp': True}
    )
    return result

### Differentiel evolution with intermediate solutions

In [11]:
initial_guess = np.array([0.0005,0.0001,0.93,0.215,0.030])

# Define a callback function to store intermediate results
intermediate_solutions = []

def callback(xk):
    intermediate_solutions.append(xk)

result = sco.minimize(
    objective_function,
    x0=initial_guess,
    args=(triplets,),
    method='TNC',
    bounds=bounds,
    callback=callback
)

DT = 2.0542848801596847e-05 back_h_simple DT>0 Ra_L<1e5 -> h = 0.01 8316.319788149174
DT = 2.18674238112726e-05 top h simple theta<=45 Ra_L -> h = 0.01 12918.602529953616
DT = 2.0572793573592207e-05 back_h_simple DT>0 Ra_L<1e5 -> h = 0.01 8328.44226747866
DT = 2.189456444057214e-05 top h simple theta<=45 Ra_L -> h = 0.01 12934.636377631696
DT = 2.05738857630422e-05 back_h_simple DT>0 Ra_L<1e5 -> h = 0.01 8328.884416246074
DT = 2.1895554539241857e-05 top h simple theta<=45 Ra_L -> h = 0.01 12935.221297423994
DT = 2.0574427423980524e-05 back_h_simple DT>0 Ra_L<1e5 -> h = 0.01 8329.103695709211
DT = 2.189604543900714e-05 top h simple theta<=45 Ra_L -> h = 0.01 12935.511305882274
DT = 2.0574708912590722e-05 back_h_simple DT>0 Ra_L<1e5 -> h = 0.01 8329.217650153687
DT = 2.1896300552270986e-05 top h simple theta<=45 Ra_L -> h = 0.01 12935.662018939332
DT = 2.0574832547026745e-05 back_h_simple DT>0 Ra_L<1e5 -> h = 0.01 8329.26770082346
DT = 5.592429022271972e-05 top h simple theta<=45 Ra_L ->

In [13]:
intermediate_solutions

[array([4.27644144e-04, 5.00000000e-05, 9.30615517e-01, 2.14993186e-01,
        3.01971322e-02]),
 array([2.03569213e-04, 5.00000000e-05, 9.32232489e-01, 2.14975188e-01,
        3.08418501e-02]),
 array([1.94319250e-04, 5.00000000e-05, 9.32299301e-01, 2.14974419e-01,
        3.08684849e-02]),
 array([1.94789238e-04, 5.00000000e-05, 9.32296012e-01, 2.14974417e-01,
        3.08671785e-02]),
 array([1.95113091e-04, 5.00000000e-05, 9.32293751e-01, 2.14974415e-01,
        3.08662806e-02]),
 array([1.94959216e-04, 5.00000000e-05, 9.32294827e-01, 2.14974416e-01,
        3.08667081e-02]),
 array([1.94979052e-04, 5.00000000e-05, 9.32294690e-01, 2.14974416e-01,
        3.08666535e-02]),
 array([1.94993609e-04, 5.00000000e-05, 9.32294599e-01, 2.14974416e-01,
        3.08666173e-02]),
 array([1.95092565e-04, 5.00000000e-05, 9.32294381e-01, 2.14974411e-01,
        3.08665207e-02]),
 array([1.94142419e-04, 5.00000000e-05, 9.32334255e-01, 2.14974158e-01,
        3.08815562e-02]),
 array([1.98946968e-

In [12]:
result

 message: Max. number of function evaluations reached
 success: False
  status: 3
     fun: 179.3495408150501
       x: [ 1.963e-04  5.000e-05  9.350e-01  2.150e-01  3.188e-02]
     nit: 12
     jac: [-2.020e+04  1.295e+05 -1.079e+01  4.318e+01 -1.102e+02]
    nfev: 606